In [1]:
!pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain langchain-chroma python-dotenv bs4 --quiet


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
# Ensure these Enviornment varaibles are set

# os.environ["LANGCHAIN_TRACING_V2"] = "<>"
# os.environ["LANGCHAIN_ENDPOINT"] = "<>"
# os.environ["LANGCHAIN_API_KEY"] = "<>"
# os.environ["OPENAI_API_KEY"] = "<>"

In [4]:
#### INDEXING: Reading Data ####

import bs4
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

In [5]:
#### INDEXING: Spliting Data ####

# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(blog_docs)

In [6]:
# to persit Chroma and load it again later

########################################
# Save
########################################

from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings(),
    persist_directory="./chroma_db"  # Specify the directory where you want to persist data
)

########################################
# Load
########################################

# from langchain_openai import OpenAIEmbeddings
# from langchain_community.vectorstores import Chroma

vectorstore = Chroma(
    persist_directory="./chroma_db",
    embedding_function=OpenAIEmbeddings()
)

retriever = vectorstore.as_retriever()

In [7]:
# Few Shot Examples
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
examples = [
    {
        "input": "Could the members of The Police perform lawful arrests?",
        "output": "what can the members of The Police do?",
    },
    {
        "input": "Jan Sindel’s was born in what country?",
        "output": "what is Jan Sindel’s personal history?",
    },
]
# We now transform these to example messages
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:""",
        ),
        # Few shot examples
        few_shot_prompt,
        # New question
        ("user", "{question}"),
    ]
)

In [11]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

generate_queries_step_back = prompt | ChatOpenAI(temperature=0) | StrOutputParser()
question = "What is task decomposition for LLM agents?"
generate_queries_step_back.invoke({"question": question})

'What is the process of breaking down tasks for LLM agents?'

In [13]:
# Response prompt
from langchain_core.runnables import RunnableLambda

response_prompt_template = """You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.

# {normal_context}
# {step_back_context}

# Original Question: {question}
# Answer:"""
response_prompt = ChatPromptTemplate.from_template(response_prompt_template)

chain = (
    {
        # Retrieve context using the normal question
        "normal_context": RunnableLambda(lambda x: x["question"]) | retriever,
        # Retrieve context using the step-back question
        "step_back_context": generate_queries_step_back | retriever,
        # Pass on the question
        "question": lambda x: x["question"],
    }
    | response_prompt
    | ChatOpenAI(temperature=0)
    | StrOutputParser()
)

chain.invoke({"question": question})

'Task decomposition for LLM agents refers to the process of breaking down complex tasks into smaller and more manageable subtasks that can be easily handled by the agent. This approach allows LLM-powered autonomous agents to effectively tackle intricate tasks by dividing them into simpler steps, thereby enhancing their performance and efficiency.\n\nOne common technique used for task decomposition in LLM agents is the Chain of Thought (CoT) method, which involves prompting the model to "think step by step" and decompose difficult tasks into smaller components. This technique enables the agent to utilize more computational resources during testing to effectively break down complex tasks.\n\nAnother advanced method is the Tree of Thoughts, which extends the CoT approach by exploring multiple reasoning possibilities at each step. This method generates multiple thoughts per step, creating a tree structure that helps in analyzing various paths to solve a problem. The search process in the T